In [3]:
# Import necessary libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from google.colab import files

# Load LFW dataset (Labeled Faces in the Wild)
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# The images and the corresponding labels (person names)
images = lfw_people.images
labels = lfw_people.target
target_names = lfw_people.target_names

# Preprocess the data
images = images.reshape(len(images), -1)  # Flatten images into 1D arrays (64x47 pixels)
images = images / 255.0  # Normalize the pixel values

# Encode labels (convert names to numbers)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# One-hot encode the labels
labels_encoded = to_categorical(labels_encoded)

# Build ANN model
model = Sequential([
    Dense(128, input_dim=images.shape[1], activation='relu'),  # Input layer (flattened images)
    Dense(64, activation='relu'),  # Hidden layer
    Dense(len(np.unique(labels)), activation='softmax')  # Output layer (number of classes)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(images, labels_encoded, epochs=20, validation_split=0.2)

# Function to detect faces and predict them from an uploaded image
def detect_faces_and_predict(image_path):
    # Load the uploaded image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load pre-trained face detection model from OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the image (with adjusted parameters)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # If no faces are detected
    if len(faces) == 0:
        print("No faces detected in the image!")
        return

    # Display the image with detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Show the image with faces marked
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # Process each detected face
    for (x, y, w, h) in faces:
        # Crop the face from the image
        face = img[y:y + h, x:x + w]

        # Resize the face to match the input size expected by the model
        face_resized = cv2.resize(face, (64, 64))
        face_resized = face_resized / 255.0  # Normalize pixel values
        face_resized = np.expand_dims(face_resized, axis=0)  # Add batch dimension

        # Predict the identity of the face
        prediction = model.predict(face_resized)
        predicted_label = np.argmax(prediction)
        person_name = label_encoder.inverse_transform([predicted_label])[0]

        # Display the predicted name on the image
        print(f"Predicted Person: {person_name}")

        # Draw the label on the image
        cv2.putText(img, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the final image with names labeled on faces
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Upload an image for testing
print("Upload a test image with faces...")
uploaded = files.upload()

# Get the uploaded file name
test_image_name = list(uploaded.keys())[0]
test_image_path = f"/content/{test_image_name}"

# Run the function to detect faces and predict their identities
detect_faces_and_predict(test_image_path)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)>